In [56]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# STAGE 1: find state URLs
url = "https://simple.wikipedia.org/wiki/List_of_U.S._states"
r = requests.get(url)
r.raise_for_status()
r

<Response [200]>

In [25]:
doc = BeautifulSoup(r.text, "html.parser")
type(doc)

tables = doc.find_all("table")
assert len(tables) == 1
tbl = tables[0]
trs = tbl.find_all("tr")
assert len(trs) >= 50

state_url = {} # key: state name, val: state URL
for tr in trs[1:]:
    th = tr.find("th")
    a = th.find("a")
    if a == None:
        continue
    url = "https://simple.wikipedia.org" + a.attrs["href"]
    state_url[a.get_text()] = url

In [35]:
# STAGE 2: download HTML for all the states
for state in list(state_url.keys()):
    url = state_url[state]
    print(state, url)
    # 1: download from Internet
    r = requests.get(url)
    r.raise_for_status()
    page_contents = r.text
    
    # 2: save it to a file
    f = open(os.path.join("states", state+".html"), "w")
    f.write(page_contents)
    f.close()

Alabama https://simple.wikipedia.org/wiki/Alabama
Alaska https://simple.wikipedia.org/wiki/Alaska
Arizona https://simple.wikipedia.org/wiki/Arizona
Arkansas https://simple.wikipedia.org/wiki/Arkansas
California https://simple.wikipedia.org/wiki/California
Colorado https://simple.wikipedia.org/wiki/Colorado
Connecticut https://simple.wikipedia.org/wiki/Connecticut
Delaware https://simple.wikipedia.org/wiki/Delaware
Florida https://simple.wikipedia.org/wiki/Florida
Georgia https://simple.wikipedia.org/wiki/Georgia_(U.S._state)
Hawaii https://simple.wikipedia.org/wiki/Hawaii
Idaho https://simple.wikipedia.org/wiki/Idaho
Illinois https://simple.wikipedia.org/wiki/Illinois
Indiana https://simple.wikipedia.org/wiki/Indiana
Iowa https://simple.wikipedia.org/wiki/Iowa
Kansas https://simple.wikipedia.org/wiki/Kansas
Kentucky https://simple.wikipedia.org/wiki/Kentucky
Louisiana https://simple.wikipedia.org/wiki/Louisiana
Maine https://simple.wikipedia.org/wiki/Maine
Maryland https://simple.wikip

In [73]:
df = pd.DataFrame()

def load_state(path):
    f = open(path)
    html = f.read()
    f.close()
    doc = BeautifulSoup(html, "html.parser")
    for tbl in doc.find_all("table"):
        if "infobox" in tbl.attrs.get("class", []):
            for row in tbl.find_all("tr"):
                cells = row.find_all(["th", "td"])
                if len(cells) == 2:
                    key = cells[0].get_text()
                    val = cells[1].get_text()
                    #print(key, ":", val)
                    name = path.split("/")[-1].split(".")[0]
                    df.loc[name, key] = val

for file_name in os.listdir("states"):
    load_state(os.path.join("states", file_name))

In [75]:
df["Largest metro"]

Ohio              Greater CincinnatiGreater Columbus(see footnot...
South Carolina                                           Greenville
Oregon                                   Portland metropolitan area
Louisiana                                       Greater New Orleans
Illinois                                            Greater Chicago
New Hampshire                                     Manchester–Nashua
North Carolina                                    Greater Charlotte
New Jersey                                         Greater New York
Mississippi                                         Greater Jackson
Nevada                                             Las Vegas Valley
Tennessee                                         Greater Nashville
Rhode Island                                     Greater Providence
Minnesota                                    Minneapolis–Saint Paul
Montana                                  Billings metropolitan area
Utah                                            